In [59]:
from typing import Callable

import numpy as np
import pandas as pd
from qiskit import Aer

from qpe import QuantumPhaseEstimation
from utils import float2_to_float10, run_job

pd.set_option("display.precision", 4)
pd.set_option("display.width", 300)

## Ensuring correctness of Quantum Phase Estimation subroutine

### QPE Algorithm 

Given a unitary operator $U$ and a quantum state $\ket \psi$ which is an eigenvector of $U$ in the form

$$U \ket \psi = e^{2 \pi \iota \theta} \ket \psi$$

We can use the algorithm to find $2^n \theta$ where $n$ is the number of qubits, and thus estimate the phase $\theta$

If a given implementation of QPE is correct, then
by setting the following

$$\theta_{\text{gate}} \coloneqq 1$$

$$U \coloneqq \begin{bmatrix} 1 & 0 \\ 0 & e^{\iota \theta} \end{bmatrix}$$

$$\ket \psi \coloneqq 1$$

We get the phase shift due to the gate as

$$2 \pi \theta = \theta_{\text{gate}} = 1$$

$$\pi = 1 / 2 \theta$$

Also, the algorithm estimates $\theta$

$$\theta = \text{output} / 2^n$$

Thus the following indicates a correct QPE implementation

$$1 / (2 \times (\text{output} / 2^n)) \approx \pi$$




In [60]:
def get_pi_estimate(n_qubits: int):
    nq = n_qubits
    circ = QuantumPhaseEstimation.get_circuit(
        counting_qubits=nq, ancillary_qubits=1, phase_angle=1
    )

    counts = run_job(circ, backend=Aer.get_backend("aer_simulator"))

    # get the count that occurred most frequently
    max_counts_result = max(counts, key=counts.get)
    max_counts_result = int(max_counts_result, 2)

    # solve for pi from the measured counts
    theta = max_counts_result / 2**n_qubits
    return 1.0 / (2 * theta)

In [61]:
pi_est = get_pi_estimate(12)
print(f"pi estimate = {pi_est}\npi diff = {abs(np.pi - pi_est)}")

pi estimate = 3.1411042944785277
pi diff = 0.0004883591112654351


## Jordan's Algorithm

This code uses a modified version of Jordan's algorithm adapted to Qiskit from [Rigetti Computing's Grove library implementation](https://github.com/rigetti/grove/blob/dc6bf6ec63e8c435fe52b1e00f707d5ce4cdb9b3/grove/alpha/jordan_gradient/jordan_gradient.py#L28)

The central algorithm is QPE, which has been implemented as a subroutine to accept any Unitary matrix as one of its inputs.

The unitary matrix used here is of the following form, and it encapsulates $\text{d}(x + h)$ as its phase.


$$U = \begin{bmatrix} e^{2 \pi \iota |f(h)|} & 0 \\ 0 & e^{2 \pi \iota |f(h)|} \end{bmatrix}$$


Since we encode $|f(h)|$ and our output must value be $< 1$, because we expect the output to be encoded as $0.b_0 b_1 \ldots b_n$; we retain the sign $s \coloneqq \mathrm{sgn}(f(h))$ and scale $f(h) \coloneqq f(h) / m$, where $m$ is the maximum value of the gradient.

The output is a binary floating point number of the form

$$0.b_0 b_1 \ldots b_n$$

which is converted to 

$$\nabla f = b_0 / 2 + b_1 / 4 + \ldots + b_n / 2^{n+1}$$

$$\frac{\mathrm{d}f(x)}{\mathrm{d}x} = s \cdot m \cdot \nabla f$$

In [83]:
def estimate_gradient(
    x: float,
    gradient_oracle: Callable[[float], float],
    gradient_max: int = 1,
    n_qubits: int = 4,
):

    fh = gradient_oracle(x)

    fh /= gradient_max
    perturb_sign = np.sign(fh)

    jordan_phase_factor = np.exp(1j * 2 * np.pi * abs(fh))

    unitary_matrix = np.array(
        [
            [jordan_phase_factor, 0],
            [0, jordan_phase_factor],
        ]
    )

    nq = n_qubits
    circ = QuantumPhaseEstimation.get_circuit(
        counting_qubits=nq, ancillary_qubits=1, unitary_matrix=unitary_matrix
    )

    # circ.draw("mpl", filename="circuit.png", style={"dpi": 250})

    counts = run_job(circ, backend=Aer.get_backend("aer_simulator"))
    # get the count that occurred most frequently
    max_counts_result = max(counts, key=counts.get)

    bf_estimate = perturb_sign * float(f"0.{max_counts_result}")
    deci_estimate = float2_to_float10(f"{bf_estimate:.16f}")
    deci_estimate *= gradient_max
    return deci_estimate

In [84]:
def quantum_gradients(
    xs: list[float] | np.ndarray,
    f: Callable[[float | np.ndarray], float | np.ndarray],
    grad_max: float,
    h: float = 1e-3,
    precision: int = 4,
):

    # we use a mock oracle since a true quantum oracle
    # is difficult to implement.
    def centre_diff_oracle(x: float) -> float:
        return float((f(x + h) - f(x - h)) / (2 * h))

    quantum_estimates = []

    for x in xs:
        quantum_estimates.append(
            estimate_gradient(x, centre_diff_oracle, grad_max, precision)
        )

    return np.array(quantum_estimates)

In [94]:
np.random.seed(42)
n_samples = 10
fn = np.sin
d1fn = np.cos
maximum_gradient = 1

xs = (np.random.random(n_samples)) * np.pi * 3

gradient_table = pd.DataFrame(
    {
        "x": xs,
        "f(x)": fn(xs),
        "f'(x) [analytical]": d1fn(xs),
        "f'(x) [quantum]": quantum_gradients(
            xs, fn, grad_max=maximum_gradient, h=0.001, precision=6
        ),
    }
)

gradient_table["error"] = abs(
    gradient_table["f'(x) [quantum]"] - gradient_table["f'(x) [analytical]"]
)
gradient_table["%error"] = abs(
    100 * gradient_table["error"] / gradient_table["f'(x) [analytical]"]
)

display(gradient_table)

,x,f(x),f'(x) [analytical],f'(x) [quantum],error,%error
0,3.5300,-0.3787,-0.9255,-0.9219,0.0037,0.3949
1,8.9603,0.4480,-0.8940,-0.8906,0.0034,0.3823
2,6.8989,0.5775,0.8164,0.8125,0.0039,0.4743
3,5.6422,-0.5980,0.8015,0.7969,0.0046,0.5796
4,1.4704,0.9950,0.1002,0.0938,0.0064,6.4249
5,1.4702,0.9949,0.1004,0.0938,0.0067,6.6357
6,0.5474,0.5205,0.8539,0.8594,0.0055,0.6450
7,8.1635,0.9525,-0.3046,-0.2969,0.0077,2.5415
8,5.6654,-0.5793,0.8151,0.8125,0.0026,0.3250
9,6.6734,0.3804,0.9248,0.9219,0.0029,0.3181


In [95]:
n_samples = 5
fn = lambda x: (10 * x) + (np.e**x) + (x**2)
d1fn = lambda x: 10 + (np.e**x) + (2 * x)
x_lim = 4
maximum_gradient = d1fn(x_lim)

xs = (np.random.rand(n_samples) - 0.5) * 2 * x_lim

gradient_table = pd.DataFrame(
    {
        "x": xs,
        "f(x)": fn(xs),
        "f'(x) [analytical]": d1fn(xs),
        "f'(x) [quantum]": quantum_gradients(
            xs, fn, grad_max=maximum_gradient, h=0.001, precision=10
        ),
    }
)

gradient_table["error"] = abs(
    gradient_table["f'(x) [quantum]"] - gradient_table["f'(x) [analytical]"]
)
gradient_table["%error"] = abs(
    100 * gradient_table["error"] / gradient_table["f'(x) [analytical]"]
)

display(gradient_table)

,x,f(x),f'(x) [analytical],f'(x) [quantum],error,%error
0,-3.8353,-23.6219,2.3509,2.3396,0.0114,0.4831
1,3.7593,94.6424,60.4360,60.4039,0.0321,0.0531
2,2.6595,47.9583,29.6088,29.6348,0.0260,0.0877
3,-2.3013,-17.6168,5.4976,5.5299,0.0324,0.5890
4,-2.5454,-18.8965,4.9876,4.9628,0.0249,0.4988
